# Read Data file

In [1]:
file = open("/users/bajajvbh/Desktop/Vaibhav/Datasets/sherlock.txt")
A = file.read()

In [2]:
len(A)

581887

# Preprocessing

In [3]:
Q = []
from re import sub
for i in A[0:10000].split(" "):
    Q.append(sub("[^A-Z0-9]","",i.upper()))

In [4]:
Q[0:10]

['PROJECT',
 'GUTENBERGS',
 'THE',
 'ADVENTURES',
 'OF',
 'SHERLOCK',
 'HOLMES',
 'BY',
 'ARTHUR',
 'CONAN']

# Arrange data in inp and op formats

In [5]:
inp = []
op = []
for i in range(4,len(Q),1):
    w0 = Q[i-4]
    w1 = Q[i-3]
    w2 = Q[i-2]
    w3 = Q[i-1]
    w4 = Q[i]
    inp.append(w0 + " " + w1 + " " + w2 + " " + w3)
    op.append(w4)  

In [6]:
import pandas as pd
W = pd.DataFrame([inp,op]).T
W.columns=["Input","Output"]
W.head()

,Input,Output
0,PROJECT GUTENBERGS THE ADVENTURES,OF
1,GUTENBERGS THE ADVENTURES OF,SHERLOCK
2,THE ADVENTURES OF SHERLOCK,HOLMES
3,ADVENTURES OF SHERLOCK HOLMES,BY
4,OF SHERLOCK HOLMES BY,ARTHUR


In [7]:
W.shape

(1683, 2)

# Find uniques

In [8]:
from numpy import unique
uniques = unique(Q)

In [9]:
len(uniques)

768

# Find lookup dictionary for uniques

In [10]:
uniques[1]

'1661LAST'

In [11]:
UL = {}
for i in range(0,len(uniques),1):
    UL[uniques[i]]=i

In [32]:
UL["AND"]

41

In [12]:
UL["SHERLOCK"]

585

# Cube declaration

In [13]:
no_lines = 1683
no_uniques = 768
seq = 4

In [14]:
from numpy import zeros
Xprov = zeros((no_lines,no_uniques,seq))

# Prepare Cube

In [15]:
for line_number,inp_seq in list(enumerate(inp)):
    for word_number,word in enumerate(inp_seq.split(" ")):
        Xprov[line_number,UL[word],word_number]=1

    Xprov[0,525,0] = 1
    Xprov[0,273,1] = 1
    Xprov[0,643,2] = 1
    Xprov[0,24,3] = 1

    Xprov[1,273,0] = 1
    Xprov[1,643,1] = 1
    Xprov[1,24,2] = 1
    Xprov[1,466,3] = 1

In [38]:
#Xprov[0,525,0]

# Prepare Y

In [39]:
Yprov = zeros((no_lines,no_uniques))

In [43]:
for wordno,word in enumerate(op):
    Yprov[wordno,UL[word]]=1

In [44]:
Yprov

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [46]:
UL["OF"]

466

In [47]:
Yprov[0,466]

1.0

# Create a NN

In [51]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.activations import softmax

In [57]:
nn = Sequential()
nn.add(LSTM(128,input_shape=(768,4)))
nn.add(Dense(768))
nn.add(Dense(768,activation='softmax'))

In [61]:
nn.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
model = nn.fit(Xprov,Yprov,validation_split=0.1,epochs=3,batch_size=21)

Epoch 1/3
73/73 [==============================] - 51s 701ms/step - loss: 5.8518 - accuracy: 0.0482 - val_loss: 7.2625 - val_accuracy: 0.0651
Epoch 2/3
73/73 [==============================] - 45s 613ms/step - loss: 5.7417 - accuracy: 0.0502 - val_loss: 7.6423 - val_accuracy: 0.0651
Epoch 3/3
73/73 [==============================] - 46s 631ms/step - loss: 5.6858 - accuracy: 0.0535 - val_loss: 7.8055 - val_accuracy: 0.0651


    Stage, Epoch, Iteration, step

    Data(chunk1,chunk2)--> epoch

    Batch size 128

    300rows(128,128,44)

# Prepare input for prediction

In [62]:
I = input("Enter words: ")
I = I.upper()

Enter words: The Adventures of Sherlock


In [73]:
Xop = zeros((1,no_uniques,seq))
for word_number,word in enumerate(I.split(" ")):
    Xop[0,UL[word],word_number]=1

R = {}
for i in range(0,len(UL.values())):
    R[list(UL.values())[i]]=list(UL.keys())[i]

predicted_val = R[np.argmax(nn.predict(Xop))]

In [95]:
predicted_val

'THE'